In [236]:
import pandas as pd
import re
import datetime


In [237]:
df = pd.read_csv("data.csv")
#df

# Column Name: Make

In [238]:
df["make"]


0         \n,Audi ,\n
1         \n,Audi ,\n
2         \n,Audi ,\n
3         \n,Audi ,\n
4         \n,Audi ,\n
             ...     
62761    \n,Volvo ,\n
62762    \n,Volvo ,\n
62763    \n,Volvo ,\n
62764    \n,Volvo ,\n
62765    \n,Volvo ,\n
Name: make, Length: 62766, dtype: object

In [239]:
df["make"]=df["make"].str.strip(',\n')
df["make"]

0         Audi 
1         Audi 
2         Audi 
3         Audi 
4         Audi 
          ...  
62761    Volvo 
62762    Volvo 
62763    Volvo 
62764    Volvo 
62765    Volvo 
Name: make, Length: 62766, dtype: object

# Column Name: model

In [240]:
df["model"]


0         \n,A3,\n
1         \n,A3,\n
2         \n,A1,\n
3         \n,A1,\n
4         \n,A3,\n
           ...    
62761    \n,S60,\n
62762    \n,S60,\n
62763    \n,S60,\n
62764    \n,S40,\n
62765    \n,S60,\n
Name: model, Length: 62766, dtype: object

In [241]:
df["model"]=df["model"].str.strip(',\n')
df["model"]

0         A3
1         A3
2         A1
3         A1
4         A3
        ... 
62761    S60
62762    S60
62763    S60
62764    S40
62765    S60
Name: model, Length: 62766, dtype: object

# Column Name: seats

In [242]:
df["seats"]

0        \n5
1        \n5
2        \n5
3        \n4
4        \n5
        ... 
62761    \n5
62762    \n5
62763    \n5
62764    \n5
62765    \n5
Name: seats, Length: 62766, dtype: object

In [243]:
df["seats"]=df["seats"].str.strip(',\n')

In [244]:
df["seats"].replace("None", "0", inplace = True) #None values may stay as None. There are no car without seat

In [245]:
df["seats"]=df["seats"].astype(int)

df["seats"].head()

0    5
1    5
2    5
3    4
4    5
Name: seats, dtype: int32

# Column Name: doors

In [246]:
df["doors"].unique()

array(['\n3', '\n5', '\n4', 'None\n', '\n2', '\n6', '\n1'], dtype=object)

In [247]:
df["doors"]=df["doors"].str.strip(',\n')
df["doors"].replace("None", "0", inplace = True) #None values may stay as None. There are no car without door
df["doors"]=df["doors"].astype(int)
df["doors"].head()

0    3
1    5
2    5
3    3
4    5
Name: doors, dtype: int32

# Column Name : seller

In [248]:
df['seller'] = df['seller'].str.lstrip("[', [' \s\S\t n ")
df['seller'] = df['seller'].str.rstrip("[', [] ' \s\S\t n")  

#seller has three not related unique values(electric, diesel , Gasoline)
df["seller"].replace("Diesel", None, inplace = True) 
df["seller"].replace("Electric", None, inplace = True) 
df["seller"].replace("Gasoline", None, inplace = True) 
df["seller"].unique()

array(['Dealer', 'Private seller', None], dtype=object)

# Column Name : previous_owner

In [249]:
df['previous_owner'] = df['previous_owner'].str.lstrip("[' \s\S\t n ',]")
df['previous_owner'] = df['previous_owner'].str.rstrip("['\s,]")
df['previous_owner']

0        None
1        None
2        None
3        None
4        None
         ... 
62761    None
62762    None
62763       1
62764    None
62765    None
Name: previous_owner, Length: 62766, dtype: object

# Column Name : engine_size

In [250]:
# kolonun ici düzenlendi, none ifade = 0 a ayarlandi.
# ancak cc biriminden dolayi int cevrilmiyor, bu yüzden de mean() vs islemlr de yapilamiyor
#??????

In [251]:

df["engine_size"]= df["engine_size"].str.strip(',\n,')
df["engine_size"]= df["engine_size"].replace("None", "0")
df["engine_size"]= df["engine_size"].str.replace("cc", "")

df["engine_size"].unique()

array(['1,984 ', '1,395 ', '1,495 ', '999 ', '1,390 ', '1,798 ', '1,197 ',
       '3,197 ', '2,773 ', '2,000 ', '1,399 ', '1,398 ', '3,189 ',
       '2,995 ', '3,199 ', '1,595 ', '2,480 ', '1,498 ', '1,781 ',
       '1,998 ', '1,500 ', '3,993 ', '3,184 ', '1,598 ', '1,800 ', '0',
       '2,393 ', '2,998 ', '4,163 ', '3,123 ', '3,195 ', '3,000 ',
       '2,976 ', '5,204 ', '1,997 ', '2,496 ', '1,968 ', '5,198 ',
       '2,894 ', '4,198 ', '1,400 ', '2,967 ', '3,597 ', '4,172 ',
       '2,671 ', '3,996 ', '1,000 ', '3,956 ', '2,700 ', '1,801 ',
       '1,995 ', '4,000 ', '2,771 ', '4,200 ', '5,998 ', '3,697 ',
       '3,200 ', '0 ', '1,799 ', '2,698 ', '4,134 ', '1,422 ', '1,191 ',
       '1,896 ', '3,001 ', '5,934 ', '1 ', '2,001 ', '1,964 ', '1,796 ',
       '1,499 ', '2,979 ', '1,599 ', '1,597 ', '4,395 ', '2,996 ', '4 ',
       '1,596 ', '2,494 ', '1,600 ', '3,246 ', '2,497 ', '4,398 ',
       '4,799 ', '4,999 ', '2,993 ', '2,171 ', '1,991 ', '2,200 ',
       '2,793 ', '4,941 ', '2,4

# Column Name : gears

In [252]:
df["gears"].unique()

array(['\n,None,\n', '\n,7,\n', '\n,6,\n', '\n,8,\n', '\n,5,\n',
       '\n,1,\n', '\n,9,\n', '\n,4,\n', '\n,2,\n', '\n,3,\n'],
      dtype=object)

In [253]:
df["gears"]=df["gears"].str.strip(',\n,')
df["gears"].unique()

array(['None', '7', '6', '8', '5', '1', '9', '4', '2', '3'], dtype=object)

bi iki koda gerek var mi pek emin degilim,
Data missing de konusabiliriz!!!
df["gears"].replace("None", "0", inplace = True)
df["gears"]=df["gears"].astype(int)

# Column Name: cylinders

In [254]:
df["cylinders"].unique()

array(['\n,4,\n', '\n,3,\n', '\n,6,\n', '\n,None,\n', '\n,5,\n',
       '\n,8,\n', '\n,10,\n', '\n,12,\n', '\n,1,\n', '\n,2,\n', '\n,7,\n'],
      dtype=object)

In [255]:
df["cylinders"]=df["cylinders"].str.strip(',\n,')
df["cylinders"].unique()

array(['4', '3', '6', 'None', '5', '8', '10', '12', '1', '2', '7'],
      dtype=object)

# Column name : fuel_consumption

In [256]:
df['fuel_consumption'] = df['fuel_consumption'].str.lstrip(" [', \s\S\t\r\n,n ' ")
df['fuel_consumption'] = df['fuel_consumption'].str.rstrip("[', '] \s\S\t\r\n,\\n  ")
df['fuel_consumption'] = df['fuel_consumption'].str.replace("'","")
df['fuel_consumption']

0                                                     None
1                                     4.9 l/100 km (comb.)
2        0 l/100 km (comb.), 0 l/100 km (city), 0 l/100...
3        4.8 l/100 km (comb.), 5.9 l/100 km (city), 4.2...
4        5 l/100 km (comb.), 6.2 l/100 km (city), 4.3 l...
                               ...                        
62761                                                 None
62762                                                 None
62763    9.1 l/100 km (comb.), 12.6 l/100 km (city), 7 ...
62764                                                 None
62765    8.4 l/100 km (comb.), 11.6 l/100 km (city), 6....
Name: fuel_consumption, Length: 62766, dtype: object

# Column Name : production_date

In [257]:
# df['production_date'] = pd.to_datetime(df['production_date'], format='%Y')

#Producton_date ve registration arasinda bag var. Yillar ayni. Burdan doldurabiliriz
df['production_date'] = df['production_date'].fillna("None")
df.loc[(df['production_date'] != 'None'), ['production_date', 'registration']]


,production_date,registration
2,2015.0,06/2015
7,2011.0,07/2011
8,2012.0,08/2012
14,2012.0,05/2012
26,2013.0,07/2013
...,...,...
62696,2007.0,03/2007
62708,2010.0,09/2010
62714,2007.0,07/2007
62719,2005.0,02/2005


# Column Name : Comfort & Convenience

In [258]:
df.rename({"\nComfort & Convenience\n":"comfort_convenience"}, inplace = True, axis = 1) # sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["comfort_convenience"] = df["comfort_convenience"].str.rstrip(" [', \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["comfort_convenience"] = df["comfort_convenience"].str.replace("'","")
df["comfort_convenience"]

0        Air conditioning, Armrest, Automatic climate c...
1        Air conditioning, Automatic climate control, E...
2        Air conditioning, Armrest, Automatic climate c...
3        Air conditioning, Armrest, Automatic climate c...
4        Air conditioning, Armrest, Electrical side mir...
                               ...                        
62761    Air conditioning, Armrest, Automatic climate c...
62762    Air conditioning, Automatic climate control, C...
62763    Air conditioning, Armrest, Automatic climate c...
62764    Air conditioning, Armrest, Automatic climate c...
62765    Air conditioning, Armrest, Automatic climate c...
Name: comfort_convenience, Length: 62766, dtype: object

# Column Name : Safety & Security

In [259]:
df.rename({"\nSafety & Security\n":"safety_security"},inplace = True, axis = 1)# sütun isminde \n değerleri olduğu için kalıcı oalrak sütun ismi değiştirildi.
df["safety_security"] = df["safety_security"].str.rstrip(" [', \s\S\t\r n ' ")
df["safety_security"] = df["safety_security"].str.lstrip(" [', '\a \s\S\t\r n ' ")
df["safety_security"] =  df["safety_security"].str.replace("'","")
df["safety_security"]

0        ABS, Alarm system, Central door lock, Central ...
1        Alarm system, Central door lock, Central door ...
2        ABS, Central door lock with remote control, Dr...
3        ABS, Alarm system, Bi-Xenon headlights, Centra...
4        ABS, Alarm system, Bi-Xenon headlights, Centra...
                               ...                        
62761    ABS, Central door lock, Central door lock with...
62762    ABS, Central door lock, Central door lock with...
62763    ABS, Central door lock, Central door lock with...
62764    ABS, Central door lock with remote control, Dr...
62765    ABS, Alarm system, Central door lock, Central ...
Name: safety_security, Length: 62766, dtype: object

# Column Name : extras

In [260]:
df["extras"] = df["extras"].str.rstrip(" [',] \s\S\t\r n ' ")
df["extras"] = df["extras"].str.lstrip(" [',] '\a \s\S\t\r n ' ")
df["extras"] =  df["extras"].str.replace("'","")
df["extras"]

0                                              Alloy wheel
1                                       Alloy wheels (16")
2                                       Alloy wheels (15")
3        Alloy wheels (17"), Shift paddles, Sport seats...
4         Alloy wheels (17"), Sport seats, Sport suspensio
                               ...                        
62761                                   Alloy wheels (15")
62762                                   Alloy wheels (15")
62763    Alloy wheels (17"), Automatically dimming inte...
62764    Alloy wheels (16"), Automatically dimming inte...
62765                                          Alloy wheel
Name: extras, Length: 62766, dtype: object

# Column Name : last_timing_belt_change

In [261]:
df['last_timing_belt_change'] = df['last_timing_belt_change'].str.lstrip(" [',] '\a \s\S\t\r n ' ")
df['last_timing_belt_change'] = df['last_timing_belt_change'].str.rstrip(" [',] '\a \s\S\t\r n ' ")

df['last_timing_belt_change'].head(25)

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23    07/2021
24       None
Name: last_timing_belt_change, dtype: object

# Column Name : registration

# Column Name : mileage

In [262]:
mileage_1 = []
for i in df['mileage']:
    result = re.search(r'\d{1,3}(?:,\d{3})*', i)
    if result:
        mileage_1.append(result.group(0))
    else:
        mileage_1.append(None)

mileage_son = pd.DataFrame(mileage_1,columns=["mileage"])

df["mileage"] = mileage_son["mileage"]


In [263]:
df.rename(columns = {'mileage':'mileage(km)'}, inplace = True)  # in place of every line, we write 'km' to column name as unit 

In [264]:
df["mileage(km)"].head(10)  

0    191,066
1     94,758
2    145,102
3    119,666
4     55,815
5     47,536
6     94,530
7    139,703
8    103,626
9    100,558
Name: mileage(km), dtype: object